# Torch.autograd를 사용한 자동 미분
- 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파
- 매개변수는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정
- pytorch에서는 torch.autograd라고 불리는 자동 미분 엔진을 사용

# 참고
https://tutorials.pytorch.kr/beginner/basics/autogradqs_tutorial.html

In [2]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad = True)
b = torch.randn(3, requires_grad = True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [3]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f66b93dd910>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f66b93dd9d0>


# 변화도 계산하기


In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1164, 0.2347, 0.0559],
        [0.1164, 0.2347, 0.0559],
        [0.1164, 0.2347, 0.0559],
        [0.1164, 0.2347, 0.0559],
        [0.1164, 0.2347, 0.0559]])
tensor([0.1164, 0.2347, 0.0559])


# 변화도 추적 멈추기
- requires_grad = True인모든 텐서들은 연산 기록을 추적하고 변화도 계산 지원
- 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우 추적이나 지원이 필요 없을 수 있음
- torch.no_grad() 블록으로 연산 추적 멈출 수 있음

In [5]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
  z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [6]:
# detach 함수를 사용하여 위의 함수와 동일한 결과를 얻음

z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


# 선택적으로 읽기(Optional Reading): 텐서 변화도와 야코비안 곱


In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph = True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph = True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph = True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
